In [1]:
import numpy as np, sys, os, holoviews as hv, scipy
sys.path.insert(0, '../') # add normal modules
from Utils import IO, plotting as plotz

root = '../Data/new4'
data = IO.DataLoader(root)

settings = {k: IO.Settings(os.path.join(root, k)) for k in data}

import multiprocessing as mp
from multiprocessing.pool import ThreadPool
processes = mp.cpu_count()

aucs = loadedData = {}
for k, v in data.items():
    tmp = os.path.join(root, k)
    setting = settings[k]
    
    tmp_worker = IO.Worker(v, settings[k])
    with ThreadPool(processes = processes) as p:
        p.map(tmp_worker, tmp_worker.idx)
        v = np.frombuffer(tmp_worker.buff, dtype = np.float64).reshape(*tmp_worker.buffshape)    
    del tmp_worker
    s = v.shape
    v = v.reshape(-1, setting.nNodes, setting.deltas // 2 - 1)
    v = np.array([(i - i.min()) / (i.max() - i.min()) for i in v])
    v = v.reshape(-1, s[-1])
    loadedData[k] = v.reshape(s)
    
    




  0%|          | 0/41 [00:00<?, ?it/s]

Extracting data...
Done
Reading settings in ../Data/new4/2019-06-03T18:17:19.448994: using json
loading graph
Reading settings in ../Data/new4/2019-06-03T20:02:14.079421: using json
loading graph
Reading settings in ../Data/new4/2019-06-03T21:50:43.035719: using json
loading graph
Reading settings in ../Data/new4/2019-06-03T23:24:28.968437: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T01:00:13.680440: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T02:26:41.454654: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T04:19:18.095176: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T06:09:10.602804: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T08:00:25.405270: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T09:15:15.042207: using json
loading graph
Reading settings in ../Data/new4/2019-06-04T10:52:36.039703: using json
loading graph


  0%|          | 0/41 [00:00<?, ?it/s]../Utils/stats.py:17: RuntimeWarning: invalid value encountered in true_divide
  kl = np.nansum(p1 * (np.log(p1 / p2)), axis = -1)
100%|██████████| 41/41 [00:08<00:00,  3.97it/s]
0it [00:00, ?it/s]
/home/casper/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [2]:
# a = next(iter(data))
# data[a]['0.8']['1']

In [3]:
def worker(sample):
    # tmp workaround
    if len(sample.shape) == 1:
        sample = sample.reshape(-1, 1)
    auc = np.zeros((len(sample), 2))
    coeffs, errors = plotz.fit(sample, func, params = fitParam)
    for nodei, c in enumerate(coeffs):
        tmp = 0
        F      = lambda x: func(x, *c)
        tmp, _ = scipy.integrate.quad(F, 0, LIMIT)
        auc[nodei, 0] = tmp
        auc[nodei, 1] = errors[nodei]
        if errors[nodei] > .1:
            print('error large')
    auc[auc < np.finfo(auc.dtype).eps ] = 0
    return auc[:, 0]
from tqdm import tqdm

double = lambda x, a, b, c, d, e, f: a + b * np.exp(-c*(x)) + d * np.exp(- e * (x-f))

double_= lambda x, b, c, d, e, f: b * np.exp(-c*(x)) + d * np.exp(- e * (x - f ))
single = lambda x, a, b, c : a + b * np.exp(-c * x)
single_= lambda x, a, b, c : a + b * np.exp(-c * x)
special= lambda x, a, b, c, d: a  + b * np.exp(- (x)**c - d)

func        = double
p0          = np.ones((func.__code__.co_argcount - 1)); # p0[0] = 0
fitParam    = dict(maxfev = int(1e6), \
                   bounds = (0, np.inf), p0 = p0,\
                   jac = 'cs')
aucs = {}
for k, v in tqdm(loadedData.items()):
    setting = settings[k]
    LIMIT = setting.deltas // 2 - 1
#     LIMIT = np.inf
    s = v.shape
    v = v.reshape(-1, *s[-2:])
    try:
        with mp.Pool(mp.cpu_count()) as p:
            auc = np.asarray(p.map(worker, v)).reshape(s[:-1])
        aucs[k] = auc
    except:
        continue

100%|██████████| 11/11 [00:10<00:00,  1.31it/s]


In [4]:
from bokeh import plotting as bp
from bokeh.models import ColumnDataSource, CustomJS
import bokeh
bp.output_notebook()


tmp_aucs = np.asarray([i for i in aucs.values()]).squeeze()
print(tmp_aucs.shape)
idx      = np.argmax(tmp_aucs, axis = -1)[:, 0]
x = []
for jdx, i in enumerate(idx):
    tmp = tmp_aucs[jdx, :2, i]
    x.append(tmp)
tmp_aucs = np.asarray(x)
# print(tmp_aucs.shape, idx.shape); assert 0

tmp_mi   = np.asarray([loadedData[i] for i in aucs.keys()]).squeeze()
tmp_mi   = tmp_mi

# setup data
buffer          = ColumnDataSource('x', 'y', \
                                   data = dict(x = [], \
                                               y = [], \
                                               color = [],\
                                              )\
                                  )
buffer_causal   = ColumnDataSource('x', 'y', \
                                   data = dict(x = [], \
                                               y = [],\
                                              )
                                  )

aucs_bokeh = ColumnDataSource('x', 'y', data = dict(x = tmp_aucs[:, 0], \
                                                    y = tmp_aucs[:, 1], \
                                                    color = bokeh.palettes.Viridis256 )\
                             )
mi_bokeh    = ColumnDataSource(data = dict(y = tmp_mi[:, 0].tolist()))
causal_bokeh= ColumnDataSource(data = dict(y = tmp_mi[:, 1].tolist()))

callback = """
//shorthands
var buff_ = buffer.data;
var mi_   = mi.data; 
var caus_ = causal.data;
var buffc_= buffer_causal.data;

//buffers
var x = [];
var y;

var indices = cb_obj.indices;
var nIndex = indices.length;

var tmp, nNodes, nTime;
var idx;

buff_['x'] = [];
buff_['y'] = [];


buffc_['x'] = [];
buffc_['y'] = [];

buff_['color'] = [];

var plt = Bokeh.Plotting;

console.log('start debug')
console.log(Bokeh);
var colors, color;
for (var ni = 0 ; ni < nIndex ; ni++){
    // get the data; build the time vector
    idx      = indices[ni];
    nNodes   = mi_['y'][idx].length;
    
    colors = [];
    for (var node = 0 ; node < nNodes ; node++){
            
        // color = Bokeh.Plotting.color(50+2*1, 100, 150);
        // colors.push(color)
        
        y     = mi_['y'][idx][node];
        nTime = y.length;
        // build time
        x = [];
        for (var t = 0 ; t < nTime ; t++){
            x.push(t);
        }
        buff_['x'].push(x);
        buff_['y'].push(y);
        // buff_['color'].push(colors)
        
        buffc_['x'].push(x);
        buffc_['y'].push(caus_['y'][idx][node]);
        }

}
buffer.change.emit();
buffer_causal.change.emit();
"""


f1 = bp.figure(tools = 'hover tap lasso_select'.split(), width = 300)
f2 = bp.figure( height = int(f1.plot_height * .5 ))
f3 = bp.figure( height = int(f1.plot_height * .5 ))

f1.scatter('x', 'y', source = aucs_bokeh, size = 10)
f2.multi_line('x', 'y', source = buffer)
f3.multi_line('x', 'y', source = buffer_causal)
JS =  CustomJS(\
             args = dict(buffer = buffer, \
                         mi     = mi_bokeh, \
                         buffer_causal = buffer_causal, \
                         causal = causal_bokeh),\
             code = callback)
aucs_bokeh.selected.js_on_change('indices', \
                                JS)
bp.show(bokeh.layouts.row(f1, bokeh.layouts.column(f2, f3)))


Loading BokehJS ...

(10, 3, 20)


In [5]:
# tmp_aucs = np.asarray([i for i in aucs.values()]).squeeze()

# tmp_aucs = np.sort(tmp_aucs, axis = -1)[..., -1:]

# print(tmp_aucs.shape)
# fig, ax = plt.subplots()
# colors = plt.cm.Spectral(np.linspace(0, 1, len(tmp_aucs)))
# [ax.scatter(*i[:2], color = colors[idx]) for idx, i in enumerate(tmp_aucs)]
# fig.show()




In [6]:
plt.cm.Spectral?

Object `plt.cm.Spectral` not found.


In [7]:
from matplotlib import style
style.reload_library()
%matplotlib inline
style.use('seaborn-poster')
fig, ax = plt.subplots(1,2)
labels = 'Underwhelming Overwhelming'.split()
for k, v in aucs.items():
    tmp = v.squeeze()
    jdx = np.argsort(tmp, axis = -1)[:, -1]
#     print([tmp[i, j] for i, j in enumerate(jdx)])
    for idx, axi in enumerate(ax):
        y = tmp[[0, idx + 1]]
        axi.scatter(*y[:, jdx[idx + 1]], color = 'blue', alpha = .2)
        axi.scatter(*tmp[[0, idx + 1]].max(-1), color = 'red', alpha = .2)
        axi.set_title(labels[idx])
        axi.set(xlabel = 'Informational impact', ylabel = 'Causal impact')
fig.subplots_adjust(wspace = .3)

axi.legend(handles = [plt.Line2D([0], [0], color = i, label = j) for i, j in zip(\
                    'blue red'.split(), 'matched max'.split())], bbox_to_anchor = (1.15, 1))
fig.show()

NameError: name 'plt' is not defined

In [ ]:
perc = np.zeros((len(aucs), 2))

# fig, ax = plt.subplots(3, 1, figsize = (10, 16))
colors = plt.cm.gist_ncar(np.linspace(0, 1, len(aucs)))

labels = "Informational impact\tUnderwhelming causal".split('\t')
for idx, (k, v) in enumerate(aucs.items()):
    
    tmp = np.argsort(v.squeeze(), axis = -1)[:, -1]
    perc[idx] = tmp[[0]] == tmp[[1, 2]]
    if not perc[idx, 0]:
        d = loadedData[k].squeeze()
        gs = dict(\
                 width_ratios = [1, 1, 1]\
                 )
        fig, ax = plt.subplots(1,3, gridspec_kw = gs)
        
        mainax = fig.add_subplot(111, \
                 frameon = 0, xticks = [], yticks = [])
        mainax.set_title(k, pad = 25)
        colors = plt.cm.Spectral(np.linspace(0, 1, settings[k].nNodes))
        g = settings[k].graph
        g = nx.node_link_graph(g)
        pos    = nx.circular_layout(g)
        subset = np.argsort(v.squeeze(), -1)[:, -2:]
        print(subset)
        for kk, axi in enumerate(ax[:2]):
            sub = subset[kk]
            [axi.plot(i, color = c, alpha = .4) for i, c in zip(d[kk, sub], colors[sub])]
            axi.set_title(labels[kk])
        nx.draw_networkx_nodes(g, pos = pos, node_color = colors, ax = ax[-1])
        nx.draw_networkx_edges(g, pos = pos, ax = ax[-1])
        ax[-1].set_aspect('equal', 'box')
        ax[-1].axes.axis('off')
        
            
        
#     vv = loadedData[k]
#     for jdx, label in enumerate(tmp):
#         axi = ax[jdx]
#         d = loadedData[k].squeeze()
#         print(d.shape)
#         axi.scatter(*d[[0, jdx + 1], label, :])
#         axi.plot(vv.squeeze()[jdx, label, :])
        
    
print(perc.mean(0), perc.std(0))


In [ ]:
fig, ax = plt.subplots()
for k, v in aucs.items():
    ax.scatter(*v[[0,1]].squeeze())
fig.savefig('test.png')

import networkx as nx
nx.from_pan
nx.from_edgelist([[0,1], [1,0]])